# Data Scientist Associate Practical Exam Submission

Use this template to complete your analysis and write up your summary for submission.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df= pd.read_csv("electric_bike_ratings_2212.csv")

In [3]:
#Validation of our dataset
df.fillna("unknown",inplace=True)
months=[]
for m in df['review_month']:
    if len(m)!=3:
        m=m[3:]
        
    months.append(m)
df.review_month=months
list_of_ages=df.reviewer_age.to_list()
while '_' in list_of_ages:
    list_of_ages.remove('-')

moyenne=[int(x) for x in list_of_ages]
moy= np.mean(moyenne)
df.reviewer_age=df.reviewer_age.replace('-', moy)
df.reviewer_age=df.reviewer_age.astype('int')
df.value_for_money = df.value_for_money.apply(lambda x: x.split('/')[0])
df.value_for_money=df.value_for_money.astype('int')

In [4]:
df.head(15)

,owned,make_model,review_month,web_browser,reviewer_age,primary_use,value_for_money,overall_rating
0,1,Nielah-Eyden,Oct,Chrome,23,Commuting,5,18.62
1,0,Nielah-Keetra,Jun,unknown,24,Commuting,4,15.55
2,0,Lunna-Keetra,Oct,Chrome,28,Commuting,5,12.76
3,1,Hoang-Keetra,Nov,IE,41,Leisure,5,17.07
4,1,Lunna-Keetra,Sep,Chrome,33,Leisure,5,12.29
5,0,Lunna-Eyden,Aug,IE,47,Commuting,5,15.28
6,0,Lunna-Keetra,Mar,Chrome,20,Leisure,3,12.28
7,1,Nielah-Keetra,Mar,Chrome,32,Commuting,7,15.92
8,1,Nielah-Eyden,Mar,Chrome,35,Commuting,4,19.39
9,1,Nielah-Eyden,Jul,Firefox,22,Commuting,6,18.38


In [5]:
df.review_month.unique()

array(['Oct', 'Jun', 'Nov', 'Sep', 'Aug', 'Mar', 'Jul', 'Apr', 'Dec',
       'Feb', 'Jan', 'May'], dtype=object)

## Task 1 
*The data set contains 1500 rows and 8 columns.*
- owned: matches the criteria with no missing values.No cleaning needed.
- make_model : matches the criteria with no missing values.No cleaning needed.
- web_browser: 6 unique values with 150 missing values. I replaced these missing values with "unknown".
- review_month: not cleaned column. I cleaned it to have month values in English short format as expected.There is no missing values.
- reviewer_age: There are 105 hypen values ("-") which considered as missing values. I replaced them with the average age.I converted this column to integer type too.
- primary_use: matches the criteria. No cleaning needed.
- value_for_money: I have transformed the column to make it have discrete values from 1 to 10 and converted it to integer type.
- overall_rating: continuous values within the range described. No cleaning needed.

After validation, the dataset contain 8 columns and 1500 rows without missing values.

## Task 2
*The category 1 has the most number of observations. nevertheless the observations are balanced across the categories of the variable "owned"*

![first visualization](1.png)

## Task 3
The overall rating ranges from 12 to 22 and the most values are between 14 and 20.

*![second visualization](2.png)*

## Task 4
*The overall rating differs according to owenership parameter. The owners are likely to give the higher overall rate than the non owners. This is shown by the boxplots below*

![third visualization](3.png)

## Task 5
*The type of this machine learning problem is **classification***

## Task 6
*I choose to fit a logistic regression model to predict the ownership.But before modeling I have performed a one hot encoding preprocessing on nominal columns.Then I splitted the data into trainig set and test set after seggregating the dataset to features(all dataset columns except "owned") and a target column ("owned"). Next the training set is scaled with standard scaler*

In [14]:
def age_categ(age):
    if age <25:
        return 0
    elif age < 40:
        return 1
    else:
        return 2
        

    

In [6]:
month_encode={"Jan":0,"Feb":1,"Mar":2,"Apr":3,"May":4,"Jun":5,"Jul":6,"Aug":7,"Sep":8,"Oct":9,"Nov":10,"Dec":11}
df.review_month = df.review_month.replace(month_encode)

In [16]:
df.reviewer_age=df.reviewer_age.apply(age_categ)


In [7]:
df.head(10)

,owned,make_model,review_month,web_browser,reviewer_age,primary_use,value_for_money,overall_rating
0,1,Nielah-Eyden,9,Chrome,23,Commuting,5,18.62
1,0,Nielah-Keetra,5,unknown,24,Commuting,4,15.55
2,0,Lunna-Keetra,9,Chrome,28,Commuting,5,12.76
3,1,Hoang-Keetra,10,IE,41,Leisure,5,17.07
4,1,Lunna-Keetra,8,Chrome,33,Leisure,5,12.29
5,0,Lunna-Eyden,7,IE,47,Commuting,5,15.28
6,0,Lunna-Keetra,2,Chrome,20,Leisure,3,12.28
7,1,Nielah-Keetra,2,Chrome,32,Commuting,7,15.92
8,1,Nielah-Eyden,2,Chrome,35,Commuting,4,19.39
9,1,Nielah-Eyden,6,Firefox,22,Commuting,6,18.38


In [17]:
# Encoding with get_dummies
df=pd.get_dummies(df,columns=['make_model', 'web_browser','primary_use'])


In [18]:
df.head()

,owned,review_month,reviewer_age,value_for_money,overall_rating,make_model_Hoang-Eyden,make_model_Hoang-Keetra,make_model_Lunna-Eyden,make_model_Lunna-Keetra,make_model_Nielah-Eyden,make_model_Nielah-Keetra,web_browser_Android,web_browser_Chrome,web_browser_Firefox,web_browser_IE,web_browser_Opera,web_browser_Safari,web_browser_unknown,primary_use_Commuting,primary_use_Leisure
0,1,9,0,5,18.62,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1,0,5,0,4,15.55,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0
2,0,9,1,5,12.76,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
3,1,10,2,5,17.07,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
4,1,8,1,5,12.29,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1


In [19]:
# data seggregation
X= df.drop('owned',axis=1)
y=df['owned']


In [20]:
# splitting data into traing set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=20, stratify=y)

In [21]:
y_test.value_counts()

1    178
0    122
Name: owned, dtype: int64

In [22]:
# fit a standard scaler with X_train
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train =sc.transform(X_train)
X_test=sc.transform(X_test)


In [23]:
# fitting the logistic regression model
from sklearn.linear_model import LogisticRegression

logreg= LogisticRegression()
logreg.fit(X_train,y_train)


LogisticRegression()

In [24]:
# predict the response for the test data
log_pred=logreg.predict(X_test)

## Task 7
*I choose the K-nearest neighbors as the comparison model*

In [25]:
# fitting KNN model
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)


KNeighborsClassifier(n_neighbors=9)

In [26]:
# predict the response for the test data
knn_pred=knn.predict(X_test)

In [27]:
from sklearn.svm import SVC
svm_class=SVC()
svm_class.fit(X_train,y_train)
svm_pred=svm_class.predict(X_test)

## Task 8
*I chose these two models because they are suitable for a binary classification problem*

## Task 9
*To evaluate the two models I used the report classification as metrics*

In [29]:
# Compare the performance of the 2 models
from sklearn.metrics import classification_report
print("LogisticRegression report")
print(classification_report(y_test,log_pred))
print("=====================================================\n")
print("KNN report")
print(classification_report(y_test,knn_pred))
print("=====================================================\n")
print(classification_report(y_test,svm_pred))

LogisticRegression report
              precision    recall  f1-score   support

           0       0.76      0.61      0.68       122
           1       0.76      0.87      0.81       178

    accuracy                           0.76       300
   macro avg       0.76      0.74      0.74       300
weighted avg       0.76      0.76      0.76       300


KNN report
              precision    recall  f1-score   support

           0       0.63      0.54      0.58       122
           1       0.71      0.78      0.75       178

    accuracy                           0.68       300
   macro avg       0.67      0.66      0.66       300
weighted avg       0.68      0.68      0.68       300


              precision    recall  f1-score   support

           0       0.71      0.55      0.62       122
           1       0.73      0.85      0.79       178

    accuracy                           0.73       300
   macro avg       0.72      0.70      0.70       300
weighted avg       0.72      0.73  

## Task 10
*The **logistic regression** model performs better than the k nearest neighbor model because it gives the best accuracy ,precison, recall and f1-score metrics. The accuracy is 76% for **logistic regression** classifier compared to 72% for **KNN** classifier*

## ✅  When you have finished...
- Publish your Workspace using the option on the left
- Check the published version of your report:
	- Can you see everything you want us to grade?
    - Are all the graphics visible?
- Review the grading rubric. Have you included everything that will be graded?
- Head back to the [Certification dashboard](https://app.datacamp.com/certification) to submit your practical exam